In [1]:
import pandas as pd

## Cleaning OpenVAS

In [2]:
nes = pd.read_excel('nessus.xlsx')

In [3]:
nes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Plugin ID                 173 non-null    int64         
 1   CVE                       36 non-null     object        
 2   CVSS v2.0 Base Score      57 non-null     float64       
 3   Risk                      56 non-null     object        
 4   Host                      173 non-null    object        
 5   Protocol                  173 non-null    object        
 6   Port                      173 non-null    int64         
 7   Name                      173 non-null    object        
 8   Synopsis                  173 non-null    object        
 9   Description               173 non-null    object        
 10  Solution                  97 non-null     object        
 11  See Also                  75 non-null     object        
 12  Plugin Output         

In [4]:
nes = nes.drop_duplicates(keep='first')
nes

,Plugin ID,CVE,CVSS v2.0 Base Score,Risk,Host,Protocol,Port,Name,Synopsis,Description,...,See Also,Plugin Output,BID,XREF,MSKB,Plugin Publication Date,Plugin Modification Date,Metasploit,Core Impact,CANVAS
0,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,http://tomcat.apache.org/connectors-doc/\nhttp...,\nThe connector listing on this port supports ...,NaN,NaN,NaN,2006-04-05,2019-11-22,NaN,NaN,NaN
1,18261,NaN,NaN,NaN,192.168.1.172,tcp,0,Apache Banner Linux Distribution Disclosure,The name of the Linux distribution running on ...,Nessus was able to extract the banner of the A...,...,NaN,\nThe Linux distribution detected was : \n - U...,NaN,NaN,NaN,2005-05-15,2022-03-21,NaN,NaN,NaN
2,48204,NaN,NaN,NaN,192.168.1.172,tcp,80,Apache HTTP Server Version,It is possible to obtain the version number of...,The remote host is running the Apache HTTP Ser...,...,https://httpd.apache.org/,\n URL : http://192.168.1.172/\n Vers...,NaN,IAVT:0001-T-0030;IAVT:0001-T-0530,NaN,2010-07-30,2023-08-17,NaN,NaN,NaN
3,134862,CVE-2020-1745,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
4,134862,CVE-2020-1938,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,52703,NaN,NaN,NaN,192.168.1.172,tcp,21,vsftpd Detection,An FTP server is listening on the remote port.,"The remote host is running vsftpd, an FTP serv...",...,http://vsftpd.beasts.org/,\n Source : 220 (vsFTPd 2.3.4)\n Version : ...,NaN,NaN,NaN,2011-03-17,2019-11-22,NaN,NaN,NaN
169,11424,NaN,NaN,NaN,192.168.1.172,tcp,80,WebDAV Detection,The remote server is running with WebDAV enabled.,WebDAV is an industry standard extension to th...,...,NaN,NaN,NaN,NaN,NaN,2003-03-20,2011-03-14,NaN,NaN,NaN
170,10150,NaN,NaN,NaN,192.168.1.172,udp,137,Windows NetBIOS / SMB Remote Host Information ...,It was possible to obtain the network name of ...,The remote host is listening on UDP port 137 o...,...,NaN,The following 7 NetBIOS names have been gather...,NaN,NaN,NaN,1999-10-12,2021-02-10,NaN,NaN,NaN
171,135860,NaN,NaN,NaN,192.168.1.172,tcp,445,WMI Not Available,WMI queries could not be made against the remo...,WMI (Windows Management Instrumentation) is no...,...,https://docs.microsoft.com/en-us/windows/win32...,Can't connect to the 'root\CIMV2' WMI namespace.,NaN,NaN,NaN,2020-04-21,2024-08-14,NaN,NaN,NaN


In [5]:
def rm_by_col(df, columns):
    df_non_nan = df.dropna(subset=columns)
    df_nan = df[df[columns[0]].isna() | df[columns[1]].isna()]

    # Drop duplicates in the non-NaN part while keeping the first occurrence based on the specified columns
    df_non_nan_no_duplicates = df_non_nan.drop_duplicates(subset=columns, keep='first')

    # Concatenate the non-NaN part with removed duplicates and the original NaN part
    df_result = pd.concat([df_non_nan_no_duplicates, df_nan], ignore_index=True)
    return df_result


In [6]:
nes = nes.drop_duplicates(subset=['CVE', 'Description', 'Port'], keep='first')
nes

,Plugin ID,CVE,CVSS v2.0 Base Score,Risk,Host,Protocol,Port,Name,Synopsis,Description,...,See Also,Plugin Output,BID,XREF,MSKB,Plugin Publication Date,Plugin Modification Date,Metasploit,Core Impact,CANVAS
0,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,http://tomcat.apache.org/connectors-doc/\nhttp...,\nThe connector listing on this port supports ...,NaN,NaN,NaN,2006-04-05,2019-11-22,NaN,NaN,NaN
1,18261,NaN,NaN,NaN,192.168.1.172,tcp,0,Apache Banner Linux Distribution Disclosure,The name of the Linux distribution running on ...,Nessus was able to extract the banner of the A...,...,NaN,\nThe Linux distribution detected was : \n - U...,NaN,NaN,NaN,2005-05-15,2022-03-21,NaN,NaN,NaN
2,48204,NaN,NaN,NaN,192.168.1.172,tcp,80,Apache HTTP Server Version,It is possible to obtain the version number of...,The remote host is running the Apache HTTP Ser...,...,https://httpd.apache.org/,\n URL : http://192.168.1.172/\n Vers...,NaN,IAVT:0001-T-0030;IAVT:0001-T-0530,NaN,2010-07-30,2023-08-17,NaN,NaN,NaN
3,134862,CVE-2020-1745,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
4,134862,CVE-2020-1938,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,52703,NaN,NaN,NaN,192.168.1.172,tcp,21,vsftpd Detection,An FTP server is listening on the remote port.,"The remote host is running vsftpd, an FTP serv...",...,http://vsftpd.beasts.org/,\n Source : 220 (vsFTPd 2.3.4)\n Version : ...,NaN,NaN,NaN,2011-03-17,2019-11-22,NaN,NaN,NaN
169,11424,NaN,NaN,NaN,192.168.1.172,tcp,80,WebDAV Detection,The remote server is running with WebDAV enabled.,WebDAV is an industry standard extension to th...,...,NaN,NaN,NaN,NaN,NaN,2003-03-20,2011-03-14,NaN,NaN,NaN
170,10150,NaN,NaN,NaN,192.168.1.172,udp,137,Windows NetBIOS / SMB Remote Host Information ...,It was possible to obtain the network name of ...,The remote host is listening on UDP port 137 o...,...,NaN,The following 7 NetBIOS names have been gather...,NaN,NaN,NaN,1999-10-12,2021-02-10,NaN,NaN,NaN
171,135860,NaN,NaN,NaN,192.168.1.172,tcp,445,WMI Not Available,WMI queries could not be made against the remo...,WMI (Windows Management Instrumentation) is no...,...,https://docs.microsoft.com/en-us/windows/win32...,Can't connect to the 'root\CIMV2' WMI namespace.,NaN,NaN,NaN,2020-04-21,2024-08-14,NaN,NaN,NaN


In [7]:
nes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 0 to 172
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Plugin ID                 169 non-null    int64         
 1   CVE                       36 non-null     object        
 2   CVSS v2.0 Base Score      57 non-null     float64       
 3   Risk                      56 non-null     object        
 4   Host                      169 non-null    object        
 5   Protocol                  169 non-null    object        
 6   Port                      169 non-null    int64         
 7   Name                      169 non-null    object        
 8   Synopsis                  169 non-null    object        
 9   Description               169 non-null    object        
 10  Solution                  96 non-null     object        
 11  See Also                  73 non-null     object        
 12  Plugin Output             1

In [8]:
nes.to_excel('new_nessus.xlsx')

## Cleaning Nessus

In [9]:
vas = pd.read_excel('openvas.xlsx')

In [10]:
vas['Port'] = vas['Port'].fillna(0).astype(int)

In [11]:
vas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   IP                              59 non-null     object 
 1   Hostname                        54 non-null     object 
 2   Port                            59 non-null     int64  
 3   Port Protocol                   53 non-null     object 
 4   QoD                             59 non-null     int64  
 5   Solution Type                   59 non-null     object 
 6   NVT Name                        59 non-null     object 
 7   Summary                         59 non-null     object 
 8   Specific Result                 59 non-null     object 
 9   NVT OID                         59 non-null     object 
 10  CVEs                            35 non-null     object 
 11  Task ID                         59 non-null     object 
 12  Task Name                       59 non

In [12]:
vas = vas[vas['IP'] == '192.168.1.172']

## Merging the DataFrames

In [13]:
nes = nes.rename(columns={"CVE": "CVEs"})

In [14]:
# Step 1: Merge DataFrames on 'CVEs' and 'Port' where both columns are not None
merged_df = pd.merge(
    vas[(vas['CVEs'].notna()) & (vas['Port'].notna())],
    nes[(nes['CVEs'].notna()) & (nes['Port'].notna())],
    on=['CVEs', 'Port'],
    how='inner'
)

# Step 2: Select rows from each DataFrame that are not in the merged_df
vas_not_merged = vas.merge(merged_df[['CVEs', 'Port']], on=['CVEs', 'Port'], how='left', indicator=True)
vas_not_merged = vas_not_merged[vas_not_merged['_merge'] == 'left_only'].drop(columns=['_merge'])

nes_not_merged = nes.merge(merged_df[['CVEs', 'Port']], on=['CVEs', 'Port'], how='left', indicator=True)
nes_not_merged = nes_not_merged[nes_not_merged['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 3: Concatenate the DataFrames that were not merged with the merged DataFrame
concatenated_df = pd.concat([merged_df, vas_not_merged, nes_not_merged], ignore_index=True)
concatenated_df

,IP,Hostname,Port,Port Protocol,QoD,Solution Type,NVT Name,Summary,Specific Result,NVT OID,...,See Also,Plugin Output,BID,XREF,MSKB,Plugin Publication Date,Plugin Modification Date,Metasploit,Core Impact,CANVAS
0,192.168.1.172,METASPLOITABLE,5432,tcp,80.0,Mitigation,SSL/TLS: SSLv3 Protocol CBC Cipher Suites Info...,This host is prone to an information disclosur...,Vulnerability was detected according to the Vu...,1.3.6.1.4.1.25623.1.0.802087,...,https://www.imperialviolet.org/2014/10/14/pood...,\nNessus determined that the remote server sup...,70574,CERT:577193,NaN,2014-10-15,2023-06-23,NaN,NaN,NaN
1,192.168.1.172,METASPLOITABLE,8009,tcp,99.0,VendorFix,Apache Tomcat AJP RCE Vulnerability (Ghostcat),Apache Tomcat is prone to a remote code execut...,"It was possible to read the file ""/WEB-INF/web...",1.3.6.1.4.1.25623.1.0.143545,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
2,192.168.1.172,METASPLOITABLE,0,NaN,80.0,Mitigation,ICMP Timestamp Reply Information Disclosure,The remote host responded to an ICMP timestamp...,The following response / ICMP packet has been ...,1.3.6.1.4.1.25623.1.0.103190,...,NaN,The difference between the local and remote cl...,NaN,CWE:200,NaN,1999-08-01,2024-05-03,NaN,NaN,NaN
3,NaN,NaN,2049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,\nThe following NFS shares could be mounted :\...,NaN,NaN,NaN,2003-03-12,2023-08-30,1.0,NaN,NaN
4,NaN,NaN,2049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,\nThe following NFS shares could be mounted :\...,NaN,NaN,NaN,2003-03-12,2023-08-30,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,NaN,NaN,45832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,\nThe following RPC services are available on ...,NaN,NaN,NaN,2002-08-24,2011-05-24,NaN,NaN,NaN
131,NaN,NaN,46298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,\nThe following RPC services are available on ...,NaN,NaN,NaN,2002-08-24,2011-05-24,NaN,NaN,NaN
132,NaN,NaN,53086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,\nThe following RPC services are available on ...,NaN,NaN,NaN,2002-08-24,2011-05-24,NaN,NaN,NaN
133,NaN,NaN,56606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,\nThe following RPC services are available on ...,NaN,NaN,NaN,2002-08-24,2011-05-24,NaN,NaN,NaN
